In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random

from google.colab import files
from datetime import datetime
from IPython.display import display

def unqFileName(base):
  now = datetime.now()
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d%m%Y_%H%M%S")
  return base + "_" + dt_string + ".csv"

def downloadOrDisplay(file, df, head=False, both=False):  
  download = ""
  while both==False and download != "Y" and download != "N":
    download = input("download? [y/n] --> ").upper()
  if both or download == "Y":
    fNm = unqFileName(file)
    df.to_csv(fNm, encoding='utf-8') 
    files.download(fNm)
  if both or download != "Y":
    if head:
      display(df.head())
    else:
      display(df)

def getVal(d, ks): #can test with d = {1:{2:{3:{4:{5:6}}}}}, ks = [0,3]
  d2 = d
  for i, k in enumerate(ks):
    if i == len(ks) - 1:
      return d2.get(k, "N/A")
    d2 = d2.get(k, {})

In [ ]:
def getAdzunaComps(search_field, country_code):                                   #returns a pandas dataframe of the top search_field companies in country_code, 5 generally, + specific locations that can be hardcoded
#TODO : i don't think i can get more than 5 for national, but i can get different locations
#documentation: https://developer.adzuna.com/overview
  len = 0
  comps = {}
  for field in ['company', 'count', 'location']:
    comps[field] = []

  locs = [[], ['US', 'Georgia', 'Madison County'],['US', 'Colorado', 'Denver'], ['US', 'California', 'San Diego County']]   #looks like if specified location, it might give between 1-5
  for loc in locs:
    locStr = ", ".join(reversed(loc))
    params = {
      'app_id': 'e25d9d98',
      'app_key': 'd3308f10a71b2ea8b049eafd60834da0',
      'what': search_field
    }
    for i, loci in enumerate(loc):
      params['location'+str(i)] = loci #location0 = loc[0]
    url = 'https://api.adzuna.com/v1/api/jobs/'+ country_code +'/top_companies/'
    resp = requests.get(url,params=params).json()['leaderboard']
    for item in resp:
      comps['company'].append(getVal(item, ['canonical_name']))
      comps['count'].append(getVal(item, ['count']))
      comps['location'].append(locStr)
  return pd.DataFrame.from_dict(comps)

In [ ]:
downloadOrDisplay("adzDSUScomps", getAdzunaComps("data science", "us"))

download? [y/n] --> n


,company,count,location
0,Deloitte,6600,
1,Takeda Pharmaceutical,5372,
2,IQVIA,2808,
3,Capital One,2106,
4,PwC,1765,
5,Careerbuilder US,1,"Madison County, Georgia, US"
6,Deloitte,67,"Denver, Colorado, US"
7,PwC,61,"Denver, Colorado, US"
8,KPMG,37,"Denver, Colorado, US"
9,ICF,33,"Denver, Colorado, US"
